In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [4]:
df= spark.read.json('Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [5]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [7]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [10]:
df.columns

['age', 'name']

In [12]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [13]:
data_schema = [StructField('age', IntegerType(),True),
              StructField('name', StringType(), True)]

In [15]:
final_struct = StructType(fields = data_schema)

In [17]:
ndf= spark.read.json('Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json',
                     schema=final_struct)

In [19]:
ndf.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [20]:
df['age']

Column<b'age'>

In [23]:
ndf.select('age')

DataFrame[age: int]

In [24]:
ndf.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [25]:
type(ndf.select('age'))

pyspark.sql.dataframe.DataFrame

In [27]:
print(df.head(2))
print(df.head(2)[0])

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]
Row(age=None, name='Michael')


In [28]:
df.withColumn('double_newage', df['age']*2).show()

+----+-------+-------------+
| age|   name|double_newage|
+----+-------+-------------+
|null|Michael|         null|
|  30|   Andy|           60|
|  19| Justin|           38|
+----+-------+-------------+



### Above is not inplace operation

In [29]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [30]:
df.withColumnRenamed('name', 'new_name').show()

+----+--------+
| age|new_name|
+----+--------+
|null| Michael|
|  30|    Andy|
|  19|  Justin|
+----+--------+



In [31]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Create temprory view to use SQL

In [32]:
df.createOrReplaceTempView('people')

In [36]:
results = spark.sql('SELECT * from people where age = 30')

In [37]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Filter data after grabbing it

In [39]:
df= spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv',
                  inferSchema=True, header=True)

In [40]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [43]:
df.head(3)[0]

Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [51]:
# filtering with SQL syntex
# df.filter("Close < 500").show() # showing all the filter data
# df.filter("Close < 500").select(['Open','Close']).show()
# df.filter(df['Close']<500).select('Volume').show()

In [52]:
# df.filter((df['Close']<200)  & (df['Open']>200)).show()

### To use the result later we have to collect the dataframe

In [54]:
result = df.filter(df['Low']==197.16).collect()
print(result) # It will retuen list 
row=result[0]
print(row)# first Item


[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]
Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)


In [57]:
row.asDict()
row.asDict()['Volume']

220441900

## Groupby and Aggregate function 

In [58]:
df=spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_DataFrames/sales_info.csv',
                 inferSchema=True, header=True)

In [59]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [60]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [61]:
df.describe().show()

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|     12|     12|                12|
|   mean|   null|   null| 360.5833333333333|
| stddev|   null|   null|250.08742410799007|
|    min|   APPL|  Chris|             120.0|
|    max|   MSFT|Vanessa|             870.0|
+-------+-------+-------+------------------+



In [62]:
df.groupBy('Company') # lot of methods can be used on this datatype , see documentation

In [65]:
df.groupBy('Company').mean().show()
# df.groupBy('Company').sum().show()
# df.groupBy('Company').max().show()
# df.groupBy('Company').count().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [66]:
df.agg({'Sales':'sum'}).show()
# df.agg({'Sales':'max'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [67]:
group_data = df.groupBy('Company')
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [68]:
from pyspark.sql.functions import countDistinct,avg, stddev

In [71]:
df.select(countDistinct('Sales')).show()
df.select(avg('Sales').alias('Average Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [72]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [73]:
from pyspark.sql.functions import format_number

In [75]:
sales_std= df.select(stddev('Sales').alias('Std'))
sales_std.select(format_number('std',2 ).alias("Final")).show()

+------+
| Final|
+------+
|250.09|
+------+



## Ordering and Sorting 

In [78]:
df.orderBy('Sales').show()
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Missing Data

In [79]:
df= spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_DataFrames/ContainsNull.csv',
                  header=True, inferSchema=True)

In [80]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [82]:
# Dropping missing data
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [85]:
df.na.drop(thresh=2).show()
# df.na.drop(how='any').show()
# df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [86]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [87]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [89]:
df.na.fill('FILL VALUE').show()
df.na.fill(0).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [90]:
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



### Fill in the mean value

In [91]:
from pyspark.sql.functions import mean

In [92]:
mean_val = df.select(mean(df['Sales'])).collect()

In [96]:
mean_sales=mean_val[0][0]

In [99]:
print(mean_sales)
df.na.fill(mean_sales, ['Sales'] ).show()

400.5
+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

